### Requirements
- Salt Lake County LIR Parcels (2019)
- TAZ Boundaries
- Owned Unit Groupings Boundaries
- Address points

In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

## Inputs

In [2]:
taz_shp = '.\\Inputs\\TAZ.shp'
parcels = '.\\Inputs\\Utah_Salt_Lake_County_Parcels_LIR.gdb\\Parcels_SaltLake_LIR_UTM12'
address_pts = '.\\Inputs\\AddressPoints.gdb\\AddressPoints_Salt_Lake'
common_areas = r'.\Inputs\salt_lake_common_areas.gdb\salt_lake_common_areas'

## Create geodatabases for output files

In [3]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "classes_HUI.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "classes_HUI.gdb")
    
scratch = os.path.join(outputs, "scratch_HUI.gdb")
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, "scratch_HUI.gdb")

## Filter Address points (used later)

In [4]:
# get address points without base address point
address_pts_lyr = arcpy.MakeFeatureLayer_management(address_pts,'address_pts_lyr')
query = (''' PtType <> 'BASE ADDRESS' ''')
arcpy.SelectLayerByAttribute_management(address_pts_lyr, 'NEW_SELECTION', query)
address_pts_no_base = arcpy.FeatureClassToFeatureClass_conversion(address_pts_lyr, scratch, '_00_address_pts_no_base')

## Prep main parcel layer

In [5]:
# select parcels within modeling area
parcels_layer = arcpy.MakeFeatureLayer_management(parcels, 'parcels') 
arcpy.SelectLayerByLocation_management(parcels_layer, 'HAVE_THEIR_CENTER_IN', taz_shp)
parcels_for_modeling = arcpy.FeatureClassToFeatureClass_conversion(parcels_layer, scratch, '_01_parcels_for_modeling')

# recalc acreage 
arcpy.CalculateField_management(parcels_for_modeling, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling, 'parcels_for_modeling_lyr') 

# house count field is a string for some reason, so this is the fix
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', """ HOUSE_CNT IS NULL """)
arcpy.CalculateField_management(parcels_for_modeling_layer, "HOUSE_CNT", """0""", "PYTHON3")
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
arcpy.AddField_management(parcels_for_modeling, 'HOUSE_CNT2', 'LONG')
arcpy.CalculateField_management(parcels_for_modeling, "HOUSE_CNT2", """int(!HOUSE_CNT!)""", "PYTHON3")

# add second built year field
arcpy.AddField_management(parcels_for_modeling, 'BUILT_YR2', 'SHORT')
arcpy.CalculateField_management(parcels_for_modeling, "BUILT_YR2", """!BUILT_YR!""", "PYTHON3")

<Result '.\\Outputs\\scratch_HUI.gdb\\_01_parcels_for_modeling'>

## More parcel prep

In [6]:
# add a field to indicate parcel type
arcpy.AddField_management(parcels_for_modeling_layer, 'TYPE_WFRC', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'SUBTYPE_WFRC', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'NOTE', 'TEXT')

<Result 'parcels_for_modeling_lyr'>

In [7]:
# count parts and rings
arcpy.AddField_management(parcels_for_modeling_layer, 'PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling_layer, 'RINGS', 'LONG')

fields = ["shape@", 'PARTS', 'RINGS']
with arcpy.da.UpdateCursor(parcels_for_modeling_layer, fields) as cursor:
    for row in cursor:
        shape = row[0]
        parts = shape.partCount
        rings = shape.boundary().partCount   
        row[1] = parts
        row[2] = rings
        cursor.updateRow(row)
        

# delete parcels with empty geometry
query = (""" PARTS =  0""")
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

In [8]:
# get a count of all parcels
count_all = arcpy.GetCount_management(parcels_for_modeling_layer)
print("# initial parcels in modeling area:\n {}".format(count_all))

# initial parcels in modeling area:
 384319


## Load Owned Unit Groupings boundaries (common areas) and summarize parcels within

In [9]:
###############
# Common Areas
###############

common_areas_lyr = arcpy.MakeFeatureLayer_management(common_areas, 'common_areas_lyr') 

# pud
query = """ SUBTYPE_WFRC IN ('pud') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_pud = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02h_ca_pud')

# multi_family
query = """ TYPE_WFRC IN ('multi_family') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_multi_family = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02i_ca_multi_family')

In [10]:
###############
# PUD
###############

tag = "single_family"
tag2 = "pud"

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_pud, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
pud_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_pud_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_pud, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_pud
join_features = pud_centroids
output_features = os.path.join(scratch, "_03a_pud_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# floor count
fieldindex = fieldmappings.findFieldMapIndex('FLOORS_CNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year mode
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year max
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(oug_sj, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_pud")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# update year built with max if mode is 0
with arcpy.da.UpdateCursor(oug_sj2, ['BUILT_YR', 'BUILT_YR2']) as cursor:
    for row in cursor:
        if row[0]  is None or row[0] < 1 or row[0] == "":
            row[0] = row[1]
        
        cursor.updateRow(row)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

20112 "single_family" parcels were selected.
364207 parcels remain...


In [11]:
###############
# multi family
###############

tag = "multi_family"

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_multi_family, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
mf_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_mf_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_multi_family, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_multi_family
join_features = mf_centroids
output_features = os.path.join(scratch, "_03b_mf_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# floor count
fieldindex = fieldmappings.findFieldMapIndex('FLOORS_CNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year mode
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year max
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_multi_family")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)


# update year built with max if mode is 0
with arcpy.da.UpdateCursor(oug_sj2, ['BUILT_YR', 'BUILT_YR2']) as cursor:
    for row in cursor:
        if row[0]  is None or row[0] < 1 or row[0] == "":
            row[0] = row[1]
        
        cursor.updateRow(row)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

50742 "multi_family" parcels were selected.
313465 parcels remain...


## Categorize parcels

In [12]:
################
# single_family
################

query = (''' PROP_TYPE IN ('103','104','111', '118', '511', '651', '888','994','998') OR PARCEL_ID IN ('28304780680000') ''')
tag="single_family"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# manual select some strays
query = (''' PARCEL_ID IN ('26253360020000','26253360050000','26253360080000','26253300160000','26253300180000',
'33072760080000','33072780070000','33102280070000','33102310150000') ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# single family - environmental degradation
query = (''' PROP_TYPE IN ('521') AND HOUSE_CNT2 > 0 ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")


# create the feature class for the parcel type
single_family = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(single_family, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(single_family, field='building_type_id', expression="1",
                                expression_type="PYTHON3")


# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

231633 "single_family" parcels were selected.
81832 parcels remain...


In [13]:
########################
# Related Parcel (SF)
########################

# select all related parcels
query= (""" PROP_TYPE IN ('957') AND PARCEL_ADD IS NOT NULL AND PARCEL_ADD <> ' '  """)
tag = "related_parcel"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
related_parcels = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_08_{}'.format(tag))

# merge related parcels into dc and dissolve if addresses match
merged = arcpy.Merge_management([single_family, related_parcels], os.path.join(scratch, '_08_merged_sf_related'))
merged_layer = arcpy.MakeFeatureLayer_management(merged, 'merged_layer')

# get parcels that have an address
query= (""" PARCEL_ADD IS NOT NULL AND PARCEL_ADD <> ' '  """)
arcpy.SelectLayerByAttribute_management(merged_layer, 'NEW_SELECTION', query)

# dissolve parcels that have addresses
dissolved_parcels = arcpy.Dissolve_management(merged_layer, os.path.join(scratch, '_08_merged_dissolve'), 
                                              ["PARCEL_ADD"], [['TOTAL_MKT_VALUE', 'SUM'],['LAND_MKT_VALUE', 'SUM'], 
                                              ['PARCEL_ACRES', 'SUM'], ['BLDG_SQFT', 'SUM'],['FLOORS_CNT', 'MAX'],
                                              ['BUILT_YR', 'MAX'], ['HOUSE_CNT2', 'SUM'],['PROP_TYPE', 'MIN'],
                                              ['TYPE_WFRC', 'FIRST'],['PARCEL_ID', 'FIRST'],
                                              ])

# invert selection and dissolve using OID
arcpy.SelectLayerByAttribute_management(merged_layer, 'SWITCH_SELECTION')
dissolved_parcels2 = arcpy.Dissolve_management(merged_layer, os.path.join(scratch, '_08_merged_dissolve2'), 
                                              ["PARCEL_ADD"], [['TOTAL_MKT_VALUE', 'SUM'],['LAND_MKT_VALUE', 'SUM'], 
                                              ['PARCEL_ACRES', 'SUM'], ['BLDG_SQFT', 'SUM'],['FLOORS_CNT', 'MAX'],
                                              ['BUILT_YR', 'MAX'], ['HOUSE_CNT2', 'SUM'],['PROP_TYPE', 'MIN'],
                                              ['TYPE_WFRC', 'FIRST'],['PARCEL_ID', 'FIRST'],
                                              ])
                                                               
sf_rp = os.path.join(scratch, '_02_single_family_with_rp')
merged2 = arcpy.Merge_management([dissolved_parcels, dissolved_parcels2], sf_rp)
merged2_layer = arcpy.MakeFeatureLayer_management(merged2, 'merged2_layer') 

# remove related parcels that did not join to others
query= (""" FIRST_TYPE_WFRC IN ('related_parcel') """)
arcpy.SelectLayerByAttribute_management(merged2_layer, 'NEW_SELECTION', query)
merged2_layer = arcpy.DeleteFeatures_management(merged2_layer)




# remove parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=merged2_layer, selection_type='NEW_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

4476 "related_parcel" parcels were selected.
77356 parcels remain...


In [14]:
# finish post processing sf with rp
sf_rp_sdf = pd.DataFrame.spatial.from_featureclass(sf_rp)

sf_rp_sdf.columns = ['OBJECTID','PARCEL_ADD','TOTAL_MKT_VALUE','LAND_MKT_VALUE',
       'PARCEL_ACRES','BLDG_SQFT','FLOORS_CNT','BUILT_YR',
       'HOUSE_CNT2','PROP_TYPE','TYPE_WFRC','PARCEL_ID',
       'SHAPE']

sf_rp_sdf = sf_rp_sdf[['OBJECTID','PARCEL_ADD','TOTAL_MKT_VALUE','LAND_MKT_VALUE',
       'PARCEL_ACRES','BLDG_SQFT','FLOORS_CNT','BUILT_YR',
       'HOUSE_CNT2','PROP_TYPE','TYPE_WFRC','PARCEL_ID',
       'SHAPE']].copy()

sf_rp_sdf['TYPE_WFRC'] = 'single_family'
sf_rp_sdf['SUBTYPE_WFRC'] = 'single_family'

sf_rp_sdf.spatial.to_featureclass(location=os.path.join(gdb, '_02_single_family_with_rp'),sanitize_columns=False)

'E:\\Projects\\REMM-Input-Data-Prep-2019\\Parcels\\2020-SaltLake\\Outputs\\classes_HUI.gdb\\_02_single_family_with_rp'

In [15]:
########################
# single_family w/ ADU
########################

query = (''' PROP_TYPE IN ('106') ''')
tag="single_family"
tag2="single_family_adu"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag2))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag2)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag2)), field='building_type_id', expression="5",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag2, count_remaining))

1439 "single_family_adu" parcels were selected.
75917 parcels remain...


In [16]:
################
# Duplex
################

tag="multi_family"
tag2="duplex"

query = (''' PROP_TYPE IN ('112', '512') ''')


# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")
# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag2))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="5",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

5399 "multi_family" parcels were selected.
70518 parcels remain...


In [17]:
################
# Residential Multi
################

# multiple buildings/residences on one parcel

tag="multi_family"
tag2="apartment"
tag3="residential_multi"

query = (''' PROP_TYPE IN ('997') ''')

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)



# create the feature class for the parcel type
features = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_02_{}'.format(tag3))


#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = features
join_features = address_pts_no_base
output_features = os.path.join(gdb, '_02_{}'.format(tag3))

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                                     match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")


# calculate the type field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag3)), field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag3)), field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag3)), field='Note', expression="'{}'".format(tag3),
                                expression_type="PYTHON3")

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag3)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag3)), field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag3, count_remaining))

533 "residential_multi" parcels were selected.
69985 parcels remain...


In [18]:
######################
# Apartments
######################

tag = "multi_family"
tag2 = "apartment"

query= (""" PROP_TYPE IN ('110','113','114', '115', '120', '150', '199') OR
            PARCEL_ID IN ('16294330010000', '16292380050000') """)


# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(parcels_for_modeling_layer, field='Note', expression="!PROP_TYPE!",
                                expression_type="PYTHON3")

# create the feature class for the parcel type
apt_commons = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_03_{}'.format(tag))


##############

query = """ PROP_TYPE IN ('504','713', '913') or PARCEL_ID IN ('16294330010000') """

# filter for common areas
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")


common_areas =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_03a_apt_commons')

# fill in holes via eliminate polygon part
apt_commons2 = arcpy.EliminatePolygonPart_management(common_areas, os.path.join(scratch, '_03b_apt_common_areas_filled'),
                                                      condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')


#=========================
# get merge apt commons
#=========================

merged_apt_commons = arcpy.Merge_management([apt_commons, apt_commons2], 
                                              os.path.join(scratch, '_03c_merged_apt_commons'))
# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(merged_apt_commons),"PARCEL_ACRES" "AREA",'', 'ACRES')   
arcpy.CalculateField_management(merged_apt_commons, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON", )

#==================================================
# summarize units attributes within common areas
#==================================================

# get parcels that are contained by apt common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=merged_apt_commons, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
apt_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03d_apt_centroids'), "INSIDE")




# use spatial join to summarize market value & acreage
target_features = merged_apt_commons
join_features = apt_centroids
output_features = os.path.join(scratch, "_03e_apt_oug_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year mode
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year max
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "CONTAINS")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!".format(tag),
                                expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_apartment")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")



#=========================
# final summary
#=========================

# update year built with max if mode is 0
with arcpy.da.UpdateCursor(oug_sj2, ['BUILT_YR', 'BUILT_YR2']) as cursor:
    for row in cursor:
        if row[0]  is None or row[0] < 1 or row[0] == "":
            row[0] = row[1]
        
        cursor.updateRow(row)

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, "_02_apartment"), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, "_02_apartment"), field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=merged_apt_commons, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=merged_apt_commons, selection_type='ADD_TO_SELECTION')


# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# select all features within merged dataset and delete
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag2, count_remaining))

5399 "apartment" parcels were selected.
64586 parcels remain...


In [19]:
#####################
# low income apartments
#####################

tag="multi_family"
tag2 = "apartment"
tag3 = "low_income_housing"

query= (""" PROP_TYPE IN ('142') """)

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the subtype field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# calculate the notes field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='Note', expression="'{}'".format(tag3),
                                expression_type="PYTHON3")

# # create the feature class for the parcel type
# features = arcpy.EliminatePolygonPart_management(parcels_for_modeling_layer, os.path.join(scratch, '_06a_{}'.format(tag3)),
#                                                       condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')

# layer = arcpy.MakeFeatureLayer_management(features, 'layer') 
# arcpy.SelectLayerByLocation_management(in_layer=layer,overlap_type="WITHIN",
#                                        select_features=features,
#                                        selection_type='NEW_SELECTION')
# layer = arcpy.DeleteFeatures_management(layer)

# create the feature class for the parcel type
li = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_06a_{}'.format(tag3))

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# recalc acreage 
arcpy.CalculateField_management(li, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = li
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_LowIncomeApts")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")


# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, "_02_LowIncomeApts"), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, "_02_LowIncomeApts"), field='building_type_id', expression="2",
                                expression_type="PYTHON3")


# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=merged_apt_commons, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=merged_apt_commons, selection_type='ADD_TO_SELECTION')


# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# select all features within merged dataset and delete
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag3, count_remaining))

0 "low_income_housing" parcels were selected.
64586 parcels remain...


In [20]:
#################
# mobile home parks
#################

tag = "multi_family"
tag2 = "mobile_home_park"

# use overlay to select mobile home parks parcels
mobile_home_parks = ".\\Inputs\\Mobile_Home_Parks.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=mobile_home_parks,
                                       selection_type='NEW_SELECTION')

query= (""" PROP_CLASS IN ('118- MOBILE HOME') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
mhp = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_07a_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(mhp, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
arcpy.CalculateField_management(mhp, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = mhp
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_mobile_home_park")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

55 "multi_family" parcels were selected.
64531 parcels remain...


## Merge the residential parcels back to together and format/process using pandas

In [21]:
# create paths
res_files = ['_02_pud','_02_single_family_with_rp', '_02_single_family_adu','_02_duplex', 
             '_02_mobile_home_park','_02_multi_family','_02_apartment','_02_residential_multi',
             '_02_LowIncomeApts']
res_files = [os.path.join(gdb, res_file) for res_file in res_files]

# merge features
rf_merged = arcpy.Merge_management(res_files, os.path.join(scratch, '_10a_Housing_Unit_Inventory'), 
                                   add_source='ADD_SOURCE_INFO')
#-----------------------------
# add cities as an attribute
#-----------------------------
cities = r'.\Inputs\Cities.shp' 
target_features = rf_merged
join_features = cities
output_features = os.path.join(scratch, '_10b_Housing_Unit_Inventory_City')
rf_merged = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="HAVE_THEIR_CENTER_IN")

#---------------------------------
# add subcounties as an attribute
#---------------------------------
subcounties = r'.\Inputs\SubCountyArea_2019.shp' 
target_features = rf_merged
join_features = subcounties
output_features = os.path.join(scratch, '_10c_Housing_Unit_Inventory_SubCounty')
rf_merged = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="HAVE_THEIR_CENTER_IN")


# convert to dataframe and export
rf_merged_df = pd.DataFrame.spatial.from_featureclass(rf_merged)
rf_merged_df = rf_merged_df[['OBJECTID','TYPE_WFRC', 'SUBTYPE_WFRC', 'PARCEL_ID', 'TOTAL_MKT_VALUE', 
                             'LAND_MKT_VALUE', 'PARCEL_ACRES', 'HOUSE_CNT', 'parcel_count', 'ap_count', 
                             'BLDG_SQFT','FLOORS_CNT','BUILT_YR','NAME','NewSA','Note','SHAPE']].copy()

rf_merged_df = rf_merged_df.rename(columns={"NAME": "CITY"})
rf_merged_df = rf_merged_df.rename(columns={"NewSA": "SUBREGION"})

In [22]:
# convert unit count columns to int
rf_merged_df.loc[(rf_merged_df['HOUSE_CNT'].isnull() == True ), 'HOUSE_CNT'] = 0
rf_merged_df.loc[(rf_merged_df['ap_count'].isnull() == True ), 'ap_count'] = 0
rf_merged_df['ap_count'] = rf_merged_df['ap_count'].astype(int)


# create new count field and calculate
rf_merged_df['UNIT_COUNT'] = rf_merged_df['ap_count']


# fix single family (non-pud)
rf_merged_df.loc[(rf_merged_df['UNIT_COUNT'] == 0) & 
                 (rf_merged_df['TYPE_WFRC'] == 'single_family'), 
                 'UNIT_COUNT'] = 1

# fix duplex
rf_merged_df.loc[(rf_merged_df['UNIT_COUNT'] == 0) & 
                 (rf_merged_df['TYPE_WFRC'] == 'duplex'), 
                 'UNIT_COUNT'] = 2

rf_merged_df.loc[(rf_merged_df['SUBTYPE_WFRC'] == 'duplex'), 'UNIT_COUNT'] = 2


# calculate the decade
rf_merged_df['BUILT_DECADE'] = 'NA'
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1840) & (rf_merged_df['BUILT_YR'] < 1850), 'BUILT_DECADE'] = "1840's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1850) & (rf_merged_df['BUILT_YR'] < 1860), 'BUILT_DECADE'] = "1850's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1860) & (rf_merged_df['BUILT_YR'] < 1870), 'BUILT_DECADE'] = "1860's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1870) & (rf_merged_df['BUILT_YR'] < 1880), 'BUILT_DECADE'] = "1870's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1880) & (rf_merged_df['BUILT_YR'] < 1890), 'BUILT_DECADE'] = "1880's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1890) & (rf_merged_df['BUILT_YR'] < 1900), 'BUILT_DECADE'] = "1890's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1900) & (rf_merged_df['BUILT_YR'] < 1910), 'BUILT_DECADE'] = "1900's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1910) & (rf_merged_df['BUILT_YR'] < 1920), 'BUILT_DECADE'] = "1910's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1920) & (rf_merged_df['BUILT_YR'] < 1930), 'BUILT_DECADE'] = "1920's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1930) & (rf_merged_df['BUILT_YR'] < 1940), 'BUILT_DECADE'] = "1930's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1940) & (rf_merged_df['BUILT_YR'] < 1950), 'BUILT_DECADE'] = "1940's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1950) & (rf_merged_df['BUILT_YR'] < 1960), 'BUILT_DECADE'] = "1950's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1960) & (rf_merged_df['BUILT_YR'] < 1970), 'BUILT_DECADE'] = "1960's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1970) & (rf_merged_df['BUILT_YR'] < 1980), 'BUILT_DECADE'] = "1970's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1980) & (rf_merged_df['BUILT_YR'] < 1990), 'BUILT_DECADE'] = "1980's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1990) & (rf_merged_df['BUILT_YR'] < 2000), 'BUILT_DECADE'] = "1990's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 2000) & (rf_merged_df['BUILT_YR'] < 2010), 'BUILT_DECADE'] = "2000's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 2010) & (rf_merged_df['BUILT_YR'] < 2020), 'BUILT_DECADE'] = "2010's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 2020) & (rf_merged_df['BUILT_YR'] < 2030), 'BUILT_DECADE'] = "2020's"

# add county
rf_merged_df['COUNTY'] = 'SALT_LAKE'

# remove data points with zero units
rf_merged_df = rf_merged_df[~(rf_merged_df['UNIT_COUNT'] == 0) & ~(rf_merged_df['HOUSE_CNT'] == 0)].copy()

In [26]:
# Final ordering and subsetting of fields
rf_merged_df = rf_merged_df.rename(columns={"parcel_count": "PARCEL_COUNT"})

rf_merged_df = rf_merged_df[['OBJECTID','PARCEL_ID','TYPE_WFRC','SUBTYPE_WFRC','NOTE',
                             'CITY','SUBREGION','COUNTY',
                             'UNIT_COUNT','PARCEL_COUNT','FLOORS_CNT',
                             'PARCEL_ACRES','BLDG_SQFT','TOTAL_MKT_VALUE',
                             'BUILT_YR','BUILT_DECADE','SHAPE']].copy()

In [27]:
# export to feature class
rf_merged_df.spatial.to_featureclass(location=os.path.join(gdb, '_04_salt_lake_housing_unit_inventory'),sanitize_columns=False)

# export to csv
del rf_merged_df['SHAPE']
rf_merged_df.to_csv(os.path.join(outputs, 'salt_lake_housing_unit_inventory.csv'), index=False)